## Step 1: Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

## Step 2: Load Prepared Data

In [ ]:
# Load the prepared dataset
df = pd.read_csv('data/prepared_data.csv', index_col=0, parse_dates=True)
print(f"Loaded {len(df)} rows of data")
print(f"Date range: {df.index.min()} to {df.index.max()}")

## Visualization 1: Demand Over Time

In [ ]:
# Plot demand over time
fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(df.index, df['demand'], linewidth=1.5, alpha=0.8, color='#2E86AB')
ax.set_title('Andhra Pradesh Electricity Demand Over Time (2015-2023)', 
             fontsize=16, fontweight='bold', pad=20)
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Energy Required (MU)', fontsize=12)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("📈 Demand shows clear seasonal patterns with peaks during summer months")

## Visualization 2: Monthly Seasonality

In [ ]:
# Monthly average demand
numeric_cols = df.select_dtypes(include=[np.number]).columns
monthly_avg = df[numeric_cols].resample('M').mean()

fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(monthly_avg.index, monthly_avg['demand'], 
        marker='o', linewidth=2, markersize=4, color='#A23B72')
ax.set_title('Monthly Average Electricity Demand', 
             fontsize=16, fontweight='bold', pad=20)
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Average Energy Required (MU)', fontsize=12)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Visualization 3: Yearly Comparison

In [ ]:
# Yearly average demand
df['year'] = df.index.year
yearly_avg = df.groupby('year')['demand'].mean()

fig, ax = plt.subplots(figsize=(12, 6))
bars = ax.bar(yearly_avg.index, yearly_avg.values, color='#F18F01', alpha=0.8)
ax.set_title('Average Electricity Demand by Year', 
             fontsize=16, fontweight='bold', pad=20)
ax.set_xlabel('Year', fontsize=12)
ax.set_ylabel('Average Energy Required (MU)', fontsize=12)
ax.grid(True, alpha=0.3, axis='y')

# Add value labels
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.1f}', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

print(f"Year-over-year growth: {((yearly_avg.iloc[-1] - yearly_avg.iloc[0]) / yearly_avg.iloc[0] * 100):.1f}%")

## Visualization 4: Monthly Pattern

In [ ]:
# Average demand by month
df['month'] = df.index.month
monthly_pattern = df.groupby('month')['demand'].mean()
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

fig, ax = plt.subplots(figsize=(12, 6))
bars = ax.bar(range(1, 13), monthly_pattern.values, color='#C73E1D', alpha=0.8)
ax.set_title('Average Electricity Demand by Month', 
             fontsize=16, fontweight='bold', pad=20)
ax.set_xlabel('Month', fontsize=12)
ax.set_ylabel('Average Energy Required (MU)', fontsize=12)
ax.set_xticks(range(1, 13))
ax.set_xticklabels(month_names)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print(f"Peak month: {month_names[monthly_pattern.idxmax()-1]} ({monthly_pattern.max():.2f} MU)")
print(f"Lowest month: {month_names[monthly_pattern.idxmin()-1]} ({monthly_pattern.min():.2f} MU)")

## Visualization 5: Temperature vs Demand Correlation

In [ ]:
# Temperature correlation
if 'temp' in df.columns:
    correlation = df['temp'].corr(df['demand'])
    
    fig, ax = plt.subplots(figsize=(10, 6))
    scatter = ax.scatter(df['temp'], df['demand'], 
                       alpha=0.5, s=20, c=df['demand'], 
                       cmap='viridis', edgecolors='black', linewidth=0.5)
    
    ax.set_title(f'Temperature vs Electricity Demand (Correlation: {correlation:.3f})', 
                 fontsize=16, fontweight='bold', pad=20)
    ax.set_xlabel('Temperature (°C)', fontsize=12)
    ax.set_ylabel('Energy Required (MU)', fontsize=12)
    ax.grid(True, alpha=0.3)
    
    plt.colorbar(scatter, ax=ax, label='Demand (MU)')
    plt.tight_layout()
    plt.show()
    
    print(f"🌡️  Strong positive correlation: Temperature increases → Demand increases")
else:
    print("Temperature data not available")

## Visualization 6: Holiday Impact

In [ ]:
# Holiday vs work day comparison
if 'Holiday' in df.columns:
    holiday_stats = df.groupby('Holiday')['demand'].agg(['mean', 'std', 'count'])
    
    fig, ax = plt.subplots(figsize=(10, 6))
    categories = holiday_stats.index
    means = holiday_stats['mean'].values
    
    bars = ax.bar(categories, means, color=['#06A77D', '#F77F00'], alpha=0.8)
    ax.set_title('Average Electricity Demand: Holiday vs Work Days', 
                 fontsize=16, fontweight='bold', pad=20)
    ax.set_xlabel('Day Type', fontsize=12)
    ax.set_ylabel('Average Energy Required (MU)', fontsize=12)
    ax.grid(True, alpha=0.3, axis='y')
    
    # Add value labels
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.1f}', ha='center', va='bottom', fontsize=11, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print(holiday_stats)
else:
    print("Holiday data not available")

## Summary Insights

In [ ]:
print("\n" + "="*60)
print("KEY INSIGHTS FROM EDA")
print("="*60)
print(f"\n📈 Overall Trend: {((yearly_avg.iloc[-1] - yearly_avg.iloc[0]) / yearly_avg.iloc[0] * 100):.1f}% growth from 2015-2023")
print(f"\n🌞 Seasonal Pattern: Peak demand in {month_names[monthly_pattern.idxmax()-1]} ({monthly_pattern.max():.2f} MU)")
print(f"                      Low demand in {month_names[monthly_pattern.idxmin()-1]} ({monthly_pattern.min():.2f} MU)")
print(f"\n🌡️  Temperature Effect: Strong correlation ({correlation:.3f}) - Higher temps = Higher demand")
print(f"\n📊 Average Demand: {df['demand'].mean():.2f} MU")
print(f"   Std Deviation: {df['demand'].std():.2f} MU")
print(f"   Range: {df['demand'].min():.2f} - {df['demand'].max():.2f} MU")
print("\n" + "="*60)